# Immune Data Embeddings

In [1]:
from config import ScImmuneConfig
from model import ScImmuneModel
from tokenizer import ScImmuneTokenizer # refactored version
import torch
import os
import shutil
from utils import generate_metadata_embeddings, generate_metadata_tokens, assign_metadata_embeddings
from gensim.models import Word2Vec
import anndata as ad
import pandas as pd
import scanpy as sc
import json

/home/s5srinivasan/immunegptenv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


## Modify tokens and embeddings

In [2]:
shutil.copy("config.json", "scimmune-model/config.json")
shutil.copy("og_model.bin", "scimmune-model/pytorch_model.bin")

'scimmune-model/pytorch_model.bin'

In [6]:
local_config = ScImmuneConfig.from_pretrained("scimmune-model") # load config locally
local_model = ScImmuneModel(local_config) # load model locally
local_tokenizer = ScImmuneTokenizer(vocab_file="vocab_with_metadata.json") # initialize tokenizer

In [12]:
new_vocab_len = len(local_tokenizer)

In [ ]:
local_model._resize_token_embeddings(new_vocab_len) # use the "_" before the method name, that actually resizes it

Embedding(60926, 512, padding_idx=0)

## Re-initialize metadata tokens with Node2Vec vectors

In [26]:
embedding_layer = local_model.get_input_embeddings()

In [34]:
with open("vocab_with_metadata.json", "r") as f:
    vocab_with_metadata_dict = json.load(f) # load this as a dict for lookup